# Imports

In [8]:
import torch 
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import pandas as pd 
import numpy as np 
from tqdm import tqdm 
from sklearn.model_selection import train_test_split

# Reading CSV as DataFrame object

In [9]:
df = pd.read_csv('USA_Housing.csv')
df = df.drop(['Address'], axis=1)
df.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
0,79545.458574,5.682861,7.009188,4.09,23086.800503,1.059034e+06
1,79248.642455,6.002900,6.730821,3.09,40173.072174,1.505891e+06
2,61287.067179,5.865890,8.512727,5.13,36882.159400,1.058988e+06
3,63345.240046,7.188236,5.586729,3.26,34310.242831,1.260617e+06
4,59982.197226,5.040555,7.839388,4.23,26354.109472,6.309435e+05


# Model class

In [10]:
class Model(nn.Module):

    def __init__(self, input_nodes=5, output_nodes=1):
        super(Model, self).__init__()
        self.seq = nn.Sequential(
            nn.Linear(input_nodes, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(),

            nn.Linear(100, 200),
            nn.BatchNorm1d(200),
            nn.ReLU(),

            nn.Linear(200, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(),

            nn.Linear(100, output_nodes)
        )

    def forward(self, x):
        return self.seq(x)

# Dataset class

In [11]:
class HousingDataset(Dataset):

    def __init__(self, df):
        df_array = np.array(df)
        self.data = torch.from_numpy(np.array(df))
        self.data = self.normalize(self.data)
        self.xvals, self.yvals = self.data[:, :-1].to(torch.float32), self.data[:, -1].to(torch.float32)
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(self.xvals, self.yvals, test_size=0.3)

    def __getitem__(self, idx):
        return self.x_train[idx], self.y_train[idx]
        
    def __len__(self):
        return self.x_train.shape[0]
    
    def normalize(self, data):
        self.means, self.stds = data.mean(0), data.std(0)
        data = (data - self.means)/self.stds
        return data

# Initializing model, dataset and loader objects

In [12]:
model = Model()
dataset = HousingDataset(df)
loader = DataLoader(dataset=dataset, batch_size=50, shuffle=True)

# Training loop

In [13]:
epochs = 20 
lr = 0.002
opt = optim.Adam(model.parameters(), lr=lr)
crit = nn.MSELoss() 

for e in range(epochs):
    model.train()
    loop = tqdm(loader, total=len(loader), leave=True, position=0)
    total_loss = 0
    for x,y in loop : 
        opt.zero_grad()
        yhat = model.forward(x)
        loss = crit(yhat.view(-1), y)
        loss.backward()
        opt.step() 

        loop.set_description(f'Epoch : [{e+1}/{epochs}]')
        loop.set_postfix(loss=loss.item())
        total_loss += loss.item()

    print(f"Epoch : {e+1} | Total loss : {total_loss}")

Epoch : [1/20]: 100%|██████████| 70/70 [00:01<00:00, 68.38it/s, loss=0.209]


Epoch : 1 | Total loss : 11.553506568074226


Epoch : [2/20]: 100%|██████████| 70/70 [00:00<00:00, 78.95it/s, loss=0.0984]


Epoch : 2 | Total loss : 8.818062424659729


Epoch : [3/20]: 100%|██████████| 70/70 [00:00<00:00, 81.84it/s, loss=0.0907]


...

Epoch : 18 | Total loss : 8.021492291241884


Epoch : [19/20]: 100%|██████████| 70/70 [00:01<00:00, 65.21it/s, loss=0.0906]


Epoch : 19 | Total loss : 7.604690071195364


Epoch : [20/20]: 100%|██████████| 70/70 [00:00<00:00, 71.64it/s, loss=0.0852]

Epoch : 20 | Total loss : 7.23213878646493


# Evaluation of model

In [16]:
def eval(model, dataset, crit):
    model.eval()
    xvals = dataset.x_test 
    yvals = dataset.y_test 
    yhat = model.forward(xvals)
    loss = crit(yhat.view(-1), yvals)
    print(f"Mean squared error : {loss.item()}")

eval(model, dataset, crit)

Mean squared error : 0.08811856061220169
